In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split

from lab1_4.fetch_and_preprocess import fetch_heart_data, preprocess_heart_data


In [ ]:
# fetch dataset 
df = fetch_heart_data()

#preprocess
X_processed, y = preprocess_heart_data(df)

# 0 - no heart disease, 1 - heart disease (no matter which type)
y_binary = df['num'].apply(lambda x: 1 if x > 0 else 0)

# --- stratified 60/20/20 split ---
test_size = 0.20
val_size = 0.20  # final fraction for validation

# 1) hold out test first
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X_processed, y_binary, test_size=test_size, stratify=y_binary, random_state=42
)

# 2) split train into train/val (adjust val size relative to remaining)
val_size_adj = val_size / (1.0 - test_size)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=val_size_adj, stratify=y_trainval, random_state=42
)

# --- ensure correct NumPy shapes for the network ---
X_train_np = np.asarray(X_train, dtype=np.float32)
y_train_np = np.asarray(y_train, dtype=np.float32).reshape(-1, 1)

X_val_np = np.asarray(X_val, dtype=np.float32)
y_val_np = np.asarray(y_val, dtype=np.float32).reshape(-1, 1)

X_test_np = np.asarray(X_test, dtype=np.float32)
y_test_np = np.asarray(y_test, dtype=np.float32)  # flat for accuracy


In [ ]:
class MLP(nn.Module):
    """
    Fully-connected network matching the previous exercise:
    - hidden_sizes: list like [16, 8] for two hidden layers with 16 and 8 units each
    - last layer uses Sigmod activation for binary classification
    - weights initialized ~N(0, weight_std), biases = bias_init 
    """
    def __init__(self, input_size, hidden_sizes, weight_std=0.01, bias_init=0.0, loss_fn=None, loss_fn_derivative=None):
        super().__init__()
        layers = []
        in_dim = input_size
        for h in hidden_sizes:
            layers.append(nn.Linear(in_dim, h))
            layers.append(nn.ReLU(inplace=True))
            in_dim = h
        layers.append(nn.Linear(in_dim, 1))
        layers.append(nn.Sigmoid())
        self.network = nn.Sequential(*layers)
        
        # Initialize weights and biases
        self.apply(lambda m: self._init_layer(m, weight_std, bias_init))
        
        def _init_layer(m, weight_std, bias_init):
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0.0, std=weight_std)
                nn.init.constant_(m.bias, bias_init)
        
        def forward(self, x):
            return self.network(x)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu
